# DELVE re-import

The raw data for delve has disappeared, and I don't know why we were blocked on downloading it again, but since DR3 has already been released, I'm just going to go with re-importing the older data set we had, and collection-ifying it.

So let's begin.

In [1]:
from dask.distributed import Client
from hats_import.pipeline import pipeline_with_client, pipeline
from hats_import import ImportArguments, CollectionArguments, VerificationArguments
import hats_import

hats_import.__version__

/astro/users/mmd11/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'0.7.1.dev4+g91d6b7dc5'

In [ ]:
args = ImportArguments.reimport_from_hats(
    "/epyc/data3/hats/catalogs/delve/delve_dr2/",
    "/epyc/data3/hats/catalogs/delve/delve_dr2_collection/",
    output_artifact_name="delve_dr2",
    
    pixel_threshold=2_500_000,
    # sort_columns="QUICK_OBJECT_ID",
    highest_healpix_order=8,
    skymap_alt_orders=[2, 4, 6],
    row_group_kwargs={"num_rows": 500_000},

    completion_email_address="delucchi@andrew.cmu.edu",
    progress_bar=True,
    simple_progress_bar=True,
)

with Client(local_directory="/epyc/data3/hats/tmp/", n_workers=10, threads_per_worker=1) as client:
    pipeline_with_client(args, client)

In [ ]:
args = (
    CollectionArguments(
        output_artifact_name="delve_dr2_collection",
        output_path="/epyc/data3/hats/catalogs/delve/",
        completion_email_address="delucchi@andrew.cmu.edu",
        progress_bar=True,
        simple_progress_bar=True,
    )
    .catalog(
        output_artifact_name="delve_dr2",
    )
    .add_margin(margin_threshold=5.0, is_default=True)
)

with Client(local_directory="/epyc/data3/hats/tmp/", n_workers=10, threads_per_worker=1) as client:
    pipeline_with_client(args, client)

In [2]:
args = VerificationArguments(
    input_catalog_path="/epyc/data3/hats/catalogs/delve/delve_dr2_collection",
    output_path="./verification/delve_dr2",
)
pipeline(args)

Loading dataset and schema.

Starting: Test hats.io.validation.is_valid_collection.
Validating collection at path /epyc/data3/hats/catalogs/delve/delve_dr2_collection ... 
Validating catalog at path /epyc/data3/hats/catalogs/delve/delve_dr2_collection/delve_dr2 ... 
Found 2129 partitions.
Approximate coverage is 71.83 % of the sky.
Validating catalog at path /epyc/data3/hats/catalogs/delve/delve_dr2_collection/delve_dr2_5arcs ... 
Found 2129 partitions.
Approximate coverage is 71.83 % of the sky.
Result: PASSED

Starting: Test that files in _metadata match the data files on disk.
Result: PASSED

Starting: Test that number of rows are equal.
	file footers vs catalog properties
	file footers vs _metadata
Result: PASSED

Starting: Test that schemas are equal, excluding metadata.
	_common_metadata vs truth
	_metadata vs truth
	file footers vs truth
Result: PASSED

Verifier results written to verification/delve_dr2/verifier_results.csv
Elapsed time (seconds): 4.07
